# Synodal Bible & Mother

## Summary

In [1]:
# Code summary of below

In [2]:
!python --version

Python 3.11.4


## Import

### Libraries

In [3]:
from backend import *

In [4]:
%matplotlib inline

### Definitions

In [5]:
texts = '../texts/fiction/utf8/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', \
             'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

In [6]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')
bibleTokenDf = pd.read_pickle('./proc/BibleVerseTokenDf.pkl')

In [7]:
# add stopword boolean, True if all attribute columns are null
motherTokenDf['stopword'] = ~motherTokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (motherTokenDf.shape[0]+1)), name='id')
motherTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

bibleTokenDf['stopword'] = ~bibleTokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (bibleTokenDf.shape[0]+1)), name='id')
bibleTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

In [14]:
motherTokenDf#.set_index(['p_id','token_id'])

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,7,13,Каждый,1_1,1_2,det,DET,каждый,None,...,Acc,None,Masc,None,Sing,None,None,None,None,False
2,1,14,18,день,1_2,1_28,obl,NOUN,день,Inan,...,Acc,None,Masc,None,Sing,None,None,None,None,False
3,1,19,22,над,1_3,1_5,case,ADP,над,None,...,None,None,None,None,None,None,None,None,None,True
4,1,23,30,рабочей,1_4,1_5,amod,ADJ,рабочий,None,...,Ins,Pos,Fem,None,Sing,None,None,None,None,False
5,1,31,40,слободкой,1_5,1_28,obl,NOUN,слободка,Inan,...,Ins,None,Fem,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86005,4436,7,13,Кто-то,1_1,1_2,nsubj,PRON,кто-то,None,...,Nom,None,None,None,None,None,None,None,None,False
86006,4436,14,21,ответил,1_2,1_0,root,VERB,ответить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
86007,4436,22,24,ей,1_3,1_2,iobj,PRON,она,None,...,Dat,None,Fem,None,Sing,3,None,None,None,False


In [31]:
bibleTokenDf[['lemma']].join(motherTokenDf[['lemma']], on='id', lsuffix='_bible', rsuffix='_mother')

,lemma_bible,lemma_mother
id,,
1,в,каждый
2,начало,день
3,сотворить,над
4,бог,рабочий
5,небо,слободка
...,...,...
676262,христос,NaN
676263,с,NaN
676264,весь,NaN


In [49]:
bibleTokenWindow = np.array([])
bibleTokenWindow = np.array([np.array(bibleTokenDf.lemma[i:i+5]) for i in range(len(bibleTokenDf.lemma)-4)])
bibleTokenWindow

array([['в', 'начало', 'сотворить', 'бог', 'небо'],
       ['начало', 'сотворить', 'бог', 'небо', 'и'],
       ['сотворить', 'бог', 'небо', 'и', 'земля'],
       ...,
       ['наш', 'иисус', 'христос', 'с', 'весь'],
       ['иисус', 'христос', 'с', 'весь', 'вы'],
       ['христос', 'с', 'весь', 'вы', 'аминь']], dtype=object)

In [50]:
motherTokenWindow = np.array([])
motherTokenWindow = np.array([np.array(motherTokenDf.lemma[i:i+5]) for i in range(len(motherTokenDf.lemma)-4)])
motherTokenWindow

array([['каждый', 'день', 'над', 'рабочий', 'слободка'],
       ['день', 'над', 'рабочий', 'слободка', 'в'],
       ['над', 'рабочий', 'слободка', 'в', 'дымный'],
       ...,
       ['хрипеть', 'несчастный', 'кто-то', 'ответить', 'она'],
       ['несчастный', 'кто-то', 'ответить', 'она', 'громкий'],
       ['кто-то', 'ответить', 'она', 'громкий', 'рыдание']], dtype=object)

In [62]:
motherTokenWindow[0]

array(['каждый', 'день', 'над', 'рабочий', 'слободка'], dtype=object)

In [64]:
order_matters = False
work_sim_count = 0
for child_window in motherTokenWindow:
    window_sim_count = 0
    for parent_window in bibleTokenWindow:
        for i in range(len(child_window)):
            if order_matters:
                if child_window[i] == parent_window[i]:
                    window_sim_count=+1
            else:
                if child_window[i] in parent_window:
                    window_sim_count=+1
    print(f"{child_window}: {window_sim_count}")

['каждый' 'день' 'над' 'рабочий' 'слободка']: 1
['день' 'над' 'рабочий' 'слободка' 'в']: 1
['над' 'рабочий' 'слободка' 'в' 'дымный']: 1


KeyboardInterrupt: 

In [78]:
def token_window(lemma_array, window_size):
    token_window_array = np.array([])
    token_window_array = np.array([np.array(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def get_sim_count(child_window_array, parent_window_array, order_matters=False):
    work_sim_count = 0
    sim_count_tuples = []
    for child_window in child_window_array:
        print(child_window)
        child_window_sim_total_count = 0
        for parent_window in parent_window_array:
            #print(parent_window)
            child_window_sim_sub_count
            for i in range(len(child_window)):
                #print(child_window[i])
                if order_matters:
                    if child_window[i] == parent_window[i]:
                        window_sim_count=+1
                else:
                    if child_window[i] in parent_window:
                        window_sim_count=+1
        print(f"{child_window}: {window_sim_count}")
        sim_count_tuples.append((child_window, window_sim_count))
    return sim_count_tuples

def dna_test_windows(*, child_lemmas, parent_lemmas, window_size):
    sim_counter = 0
    sim_list = np.array([])
    parent_window_array = token_window(parent_lemmas, window_size)
    child_window_array = token_window(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    return sim_list

In [79]:
%%time
dna_test_windows(child_lemmas=motherTokenDf.lemma[:10], parent_lemmas=bibleTokenDf.lemma, window_size=5)

['каждый' 'день' 'над' 'рабочий' 'слободка']
['каждый' 'день' 'над' 'рабочий' 'слободка']: 1
['день' 'над' 'рабочий' 'слободка' 'в']
['день' 'над' 'рабочий' 'слободка' 'в']: 1
['над' 'рабочий' 'слободка' 'в' 'дымный']
['над' 'рабочий' 'слободка' 'в' 'дымный']: 1
['рабочий' 'слободка' 'в' 'дымный' 'масляный']
['рабочий' 'слободка' 'в' 'дымный' 'масляный']: 1
['слободка' 'в' 'дымный' 'масляный' 'воздух']
['слободка' 'в' 'дымный' 'масляный' 'воздух']: 1
['в' 'дымный' 'масляный' 'воздух' 'дрожать']
['в' 'дымный' 'масляный' 'воздух' 'дрожать']: 1
CPU times: user 1min 36s, sys: 53 ms, total: 1min 36s
Wall time: 1min 36s


[(array(['каждый', 'день', 'над', 'рабочий', 'слободка'], dtype=object), 1),
 (array(['день', 'над', 'рабочий', 'слободка', 'в'], dtype=object), 1),
 (array(['над', 'рабочий', 'слободка', 'в', 'дымный'], dtype=object), 1),
 (array(['рабочий', 'слободка', 'в', 'дымный', 'масляный'], dtype=object), 1),
 (array(['слободка', 'в', 'дымный', 'масляный', 'воздух'], dtype=object), 1),
 (array(['в', 'дымный', 'масляный', 'воздух', 'дрожать'], dtype=object), 1)]

In [15]:
bibleTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,В,1_1,1_2,case,ADP,в,None,...,None,None,None,None,None,None,None,None,None,True
2,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,...,Loc,None,Neut,None,Sing,None,None,None,None,False
3,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
4,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
5,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,...,Acc,None,Neut,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676262,37110,32,38,Христа,1_5,1_4,flat:name,PROPN,христос,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False
676263,37110,39,41,со,1_6,1_8,case,ADP,с,None,...,None,None,None,None,None,None,None,None,None,True
676264,37110,42,47,всеми,1_7,1_8,det,DET,весь,None,...,Ins,None,None,None,Plur,None,None,None,None,False


How do Gor'kii's *Mother* and the Synodal Bible overlap? 
windows of 3-5 words with 1-2+ of overlap